### SQL(SQLAlchemy)

**SQLAlchemy** Python 프로그래밍 언어용 오픈소스 SQL 툴킷이자 객체 관계 매퍼(ORM).<Br>
대화내용을 어플리케이션이 종료되어도 저장해야 하는 경우 유용.

In [1]:
from dotenv import load_dotenv

load_dotenv()

from langchain_teddynote import logging

logging.langsmith("SQLAlchemy")

LangSmith 추적을 시작합니다.
[프로젝트명]
SQLAlchemy


### 사용방법

storage를 사용하려면 다음 2가지 필요:

1. Session Id - 사용자 이름, 이메일, 채팅 ID 등과 같은 세션의 고유 식별자.

2. Connection string - 데이터베이스 연결을 지정하는 문자열. 이 문자열은 SQLAlchemy의 create_engine 함수에 전달.

- SQLChatMessageHistory 클래스를 사용하여 대화 메시지 기록을 관리.
- session_id와 connection_string을 인자로 전달하여 SQLChatMessageHistory 인스턴스를 생성.
- add_user_message 메서드를 사용하여 사용자 메시지를 대화 기록에 추가.
- add_ai_message 메서드를 사용하여 AI 메시지를 대화 기록에 추가.

In [5]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 문자열 전달
# 유저별로 세션 아이디를 다르게 줄 수 있다.
chat_message_history = SQLChatMessageHistory(
    session_id = 'sql_history', connection = 'sqlite:///sqlite.db'
)

In [6]:
# 사용자 메시지 추가
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!"
)

# AI 메세지 추가
chat_message_history.add_ai_message("안녕 테디, 만나서 반가워, 나도 잘 부탁해!")

In [7]:
# 채팅 메시지 기록의 메세지들
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워. 내이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!'),
 AIMessage(content='안녕 테디, 만나서 반가워, 나도 잘 부탁해!')]

In [10]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        # 시스템 메세지 설정하여 어시스턴스 역할 정의
        ("system", "You are a helpful assistant."),
        # 이전 대화 내용을 포함하기 위해 Placeholder 추가.
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"), # 사용자의 질문을 입력받는 메시지 설정
    ]
)

chain = prompt | ChatOpenAI(model_name = 'gpt-4o-mini', temperature=0) | StrOutputParser()

In [12]:
# sqlite.db에서 대화내용 가져오는 함수
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name = user_id,
        session_id = conversation_id,
        connection='sqlite:///sqlite.db'
    )

config_fields 설정. 대화정보를 조회할 때 참고 정보로 활용.
- user_id: 사용자 ID
- conversation_id: 대화 ID

In [14]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id = "user_id",
        annotation= str,
        name = "User ID",
        description="Unique identifier for a user.",
        default = "",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id = "conversation_id",
        annotation= str,
        name = "Conversation ID",
        description="Unique identifier for a conversation.",
        default = "",
        is_shared=True,
    )
]

In [22]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key = "question",
    history_messages_key= "chat_history",
    history_factory_config=config_fields,
)

In [23]:
# "configurable" 키 아래에 "user_id", "conversation_id" key-value 쌍으로 설정.
config = {"configurable": {"user_id": "user1", "conversation_id" : "conversation1"}}

질문에 이름을 물어보는 질문. 이전에 대화정보가 있다면 올바르게 답 출력
- chain_with_history 객체는 invoke 메서드를 호출하여 질문에 대한 답변 생성.
- invoke 메서드에는 질문 딕셔너리와 config 설정이 전달.

In [24]:
response = chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 테디야"}, config)

In [25]:
print(response)

안녕하세요, 테디! 반가워요. 어떻게 도와드릴까요?


In [26]:
response_name = chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)

In [27]:
print(response_name)

당신의 이름은 테디입니다! 맞나요?


SQLite에 저장된 user2의 대화내용에서 이름에 관한 정보가 없기 때문에 답변하지 못하는 모습.

In [28]:
config = {"configurable": {"user_id": "user2", "conversation_id" : "conversation1"}}
response_user2 = chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)
print(response_user2)

죄송하지만, 당신의 이름을 알 수 없습니다. 당신의 이름을 알려주시면 좋겠습니다!
